# Agentic Framework - One-Click Colab Deployment

**Prerequisites:** GPU runtime required (Runtime -> Change runtime type -> **T4 GPU**)

| Step | Cell | What it does |
|------|------|--------------|
| 1 | **Deploy** | Clones repo, installs everything, starts all services, creates ngrok tunnels |
| 2 | **Diagnostics** | On-demand health check of all services |
| 3 | **Recovery** | Restart individual crashed services |
| 4 | **Logs** | View service logs for debugging |

**Just click Runtime -> Run all** and wait ~5 minutes.

In [ ]:
# ============================================================
# CELL 1: FULL DEPLOYMENT (Run this - everything is automated)
# ============================================================
# This cell clones the repo, installs all dependencies, starts
# all 5 microservices + Ollama + Dashboard, and creates ngrok
# tunnels for external access. Takes ~5 minutes on a T4 GPU.
#
# After completion, the watchdog loop keeps services alive.
# Your public URLs will be printed at the end.
# ============================================================

import os, subprocess, sys

REPO_URL = "https://github.com/landonking-gif/ai_final.git"
INSTALL_DIR = "/content/ai_final"

# Clone or update repo
if os.path.exists(INSTALL_DIR):
    print("Repo exists - pulling latest...")
    subprocess.run(["git", "-C", INSTALL_DIR, "pull"], capture_output=False)
else:
    print(f"Cloning {REPO_URL}...")
    subprocess.run(["git", "clone", REPO_URL, INSTALL_DIR], capture_output=False)

# Add deploy/ to Python path so we can import from it later
deploy_dir = f"{INSTALL_DIR}/deploy"
if deploy_dir not in sys.path:
    sys.path.insert(0, deploy_dir)

# Run the deployment script
os.chdir(deploy_dir)
print(f"\nStarting deployment from {deploy_dir}...\n")
!python colab_deploy.py

In [ ]:
# ============================================================
# CELL 2: DIAGNOSTICS - Run anytime to check system health
# ============================================================

import sys
if "/content/ai_final/deploy" not in sys.path:
    sys.path.insert(0, "/content/ai_final/deploy")

from colab_deploy import diagnose
diagnose()

In [ ]:
# ============================================================
# CELL 3: RECOVERY - Restart a specific crashed service
# ============================================================
# Valid service keys:
#   code_exec, memory_service, subagent_manager,
#   mcp_gateway, orchestrator, ollama
#
# Uncomment the service you want to restart:

import sys
if "/content/ai_final/deploy" not in sys.path:
    sys.path.insert(0, "/content/ai_final/deploy")

from colab_deploy import recover_service

# recover_service("orchestrator")
# recover_service("memory_service")
# recover_service("subagent_manager")
# recover_service("mcp_gateway")
# recover_service("code_exec")
# recover_service("ollama")

In [ ]:
# ============================================================
# CELL 4: LOGS - View service logs for debugging
# ============================================================
# Usage:
#   show_logs()                      -> master deployment log
#   show_logs("orchestrator")        -> specific service
#   show_logs("orchestrator", 100)   -> last 100 lines
#
# Valid keys: orchestrator, memory_service, subagent_manager,
#   mcp_gateway, code_exec, ollama, chromadb, minio, dashboard

import sys
if "/content/ai_final/deploy" not in sys.path:
    sys.path.insert(0, "/content/ai_final/deploy")

from colab_deploy import show_logs

# show_logs()                     # master log
show_logs("orchestrator", 50)     # orchestrator last 50 lines